In [113]:
### this script is for
### select kernel named as FICTURE

import spatialdata as sd
import spatialdata_io as sd_io
import os

## Set paths 

In [128]:
visiumHD_output = '/home/sjcho/projects/ATLAS_visiumHD/lung_HN00227429_20241003/1.preprocessing/Lung_tumor_HN00227429/outs'
dataset_id = 'Lung_Tumor_HN00227429'
spatula_output = '/home/sjcho/projects/ATLAS_visiumHD/lung_HN00227429_20241003/5.spatula/out/5.2.run_spatula/HN00227429_Lung_Tumor/test.tsv.gz'

save_path = '/home/sjcho/projects/ATLAS_visiumHD/lung_HN00227429_20241003/6.merge_factor_manually/outs/6.1.merge_spot/'
sample_name = 'HN00227429_Lung_Tumor_12um'

In [116]:
# add _ in front of the file name
# but why this is need? I don't know
# maybe only the spatial data developers knows
if not os.path.exists(visiumHD_output + '/_feature_slice.h5'):
    os.rename(visiumHD_output + '/feature_slice.h5', visiumHD_output + '/_feature_slice.h5')

In [118]:
sd = sd_io.visium_hd(path = visiumHD_output, bin_size = 2, dataset_id=dataset_id)
# becuase sd_io.visium_hd try to open feature_slice.h5 in outs folder with prefix of 'dataset_id + _
# so.... because of _, please change feature_slice.h5 to dataset_id + _ + feature_slice.h5
# or simply add _ in front of feature_slice.h5 and set dataset_id = ''

/home/sjcho/.conda/envs/FICTURE/lib/python3.11/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/tmp/ipykernel_50146/1601477502.py:1: UserWarning: No full resolution image found. If incorrect, please specify the path in the `fullres_image_file` parameter when calling the `visium_hd` reader function.
  sd = sd_io.visium_hd(path = visiumHD_output, bin_size = 2)


In [119]:
spatula_out = pd.read_csv(spatula_output, compression='gzip', sep='\t')
spatula_out.columns = ['#barcode_idx', 'X', 'Y', 'factor', 'assign_probability']
spatula_out.index = spatula_out['#barcode_idx']
spatula_out = spatula_out.loc[sd.tables['square_002um'].obs.index] # only remain barcodes in filtered matrix
spatula_out = spatula_out.sort_values(['Y', 'X']) # sort  by Y then X (X is for tie break)

coordinates = spatula_out[['#barcode_idx', 'X', 'Y']]
coordinates.index = coordinates['#barcode_idx']
coordinates.drop('#barcode_idx', axis=1, inplace=True)

factors = spatula_out[['#barcode_idx', 'factor', 'assign_probability']]
factors.index = factors['#barcode_idx']
factors.drop('#barcode_idx', axis=1, inplace=True)

/tmp/ipykernel_50146/2432565994.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  coordinates.drop('#barcode_idx', axis=1, inplace=True)
/tmp/ipykernel_50146/2432565994.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  factors.drop('#barcode_idx', axis=1, inplace=True)


In [121]:
# sorted index
sorted_idx = factors.index

# sort sparse count matrix
reordered_X = sd.tables['square_002um'].X[sd.tables['square_002um'].obs.index.get_indexer(sorted_idx)]

# Aggregation

In [122]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from multiprocessing import Pool
from functools import partial

def process_kernel_batch(batch_data, expression_mat, count_features, point_assignments, unique_integers, coordinates):
    """
    Process a batch of kernels at once to reduce overhead
    """
    start_idx, kernel_batch = batch_data
    batch_results = []
    
    for i, kernel in enumerate(kernel_batch):
        mask = (point_assignments == kernel).all(axis=1)
        
        if not np.any(mask):
            batch_results.append((start_idx + i, None, None, None))
            continue
            
        # Calculate sum for expression_mat
        expression_sum = np.sum(expression_mat[mask], axis=0)
        
        # Get kernel values and calculate counts
        kernel_values = count_features[mask]
        counts = np.zeros(len(unique_integers) + 2)  # +2 for NaN and grid count
        counts[0] = np.sum(np.isnan(kernel_values))
        
        # Vectorized counting for all unique integers
        for j, val in enumerate(unique_integers):
            counts[j + 1] = np.sum(kernel_values == val)

        counts[-1] = np.sum(mask)
        # Calculate average of actual coordinates instead of grid positions
        kernel_coord = np.mean(coordinates[mask], axis=0)
        
        batch_results.append((start_idx + i, expression_sum, counts, kernel_coord))
    
    return batch_results

def aggregate_features_with_counts_batch(coordinates, expression_mat, count_features, 
                                       kernel_size=12, batch_size=5000, n_cores=None, 
                                       show_progress=True):
    """
    Batch-processed version of feature aggregation
    
    Parameters:
    - coordinates: Nx2 array of (x,y) coordinates
    - expression_mat: NxF matrix for expression
    - count_features: Nx1 array for FICTURE output. containing integers and NaN values to be counted.
    - kernel_size: size of the square kernel (default: 6)
    - batch_size: number of kernels to process in each batch (default: 100)
    - n_cores: number of CPU cores to use (default: None, uses all available cores)
    - show_progress: whether to show progress bar (default: True)
    """
    # Get unique integer values (excluding NaN)
    unique_integers = np.unique(count_features[~np.isnan(count_features)]).astype(int)
    unique_integers.sort()
    
    # Find spatial bounds
    x_min, y_min = np.min(coordinates, axis=0)
    x_max, y_max = np.max(coordinates, axis=0)
    
    # Assign points to kernels using vectorized operations
    point_assignments = np.floor((coordinates - [x_min, y_min]) / kernel_size).astype(int)
    unique_kernels = np.unique(point_assignments, axis=0)
    
    # Initialize output arrays
    n_kernels = len(unique_kernels)
    aggregated_expression_mat = np.zeros((n_kernels, expression_mat.shape[1]))
    count_features_dist = np.zeros((n_kernels, len(unique_integers) + 2)) # +2 for NaN and grid count
    kernel_coordinates = np.zeros((n_kernels, 2))
    
    if show_progress:
        print(f"Processing {n_kernels} kernels in batches of {batch_size} using {n_cores if n_cores else 'all available'} cores...")
    
    # Create batches
    n_batches = (n_kernels + batch_size - 1) // batch_size
    batches = []
    for i in range(n_batches):
        start_idx = i * batch_size
        end_idx = min(start_idx + batch_size, n_kernels)
        batches.append((start_idx, unique_kernels[start_idx:end_idx]))
    
    # Prepare partial function with fixed arguments
    process_func = partial(process_kernel_batch,
                         expression_mat=expression_mat,
                         count_features=count_features,
                         point_assignments=point_assignments,
                         unique_integers=unique_integers,
                         coordinates=coordinates)  # Added coordinates parameter
    
    # Process batches in parallel
    with Pool(processes=n_cores) as pool:
        # Create iterator with progress bar if requested
        if show_progress:
            iterator = tqdm(batches, total=n_batches, desc="Processing kernel batches")
        else:
            iterator = batches
        
        # Process all batches
        all_results = pool.imap(process_func, iterator)
        
        # Collect results
        for batch_results in all_results:
            for i, expression_sum, counts, kernel_coord in batch_results:
                if expression_sum is not None:
                    aggregated_expression_mat[i] = expression_sum
                    count_features_dist[i] = counts
                    kernel_coordinates[i] = kernel_coord
    
    if show_progress:
        print("Processing complete!")
    
    return aggregated_expression_mat, count_features_dist, kernel_coordinates, ['NaN'] + list(unique_integers)

In [123]:
agg_mean, count_dist, kernel_coords, value_labels = aggregate_features_with_counts_batch(
    coordinates, reordered_X, factors.iloc[:, 0].values, kernel_size=12,
    show_progress=True, n_cores = 64, batch_size=5000
)

Processing 250669 kernels in batches of 5000 using 64 cores...


Processing kernel batches: 100%|██████████| 51/51 [07:39<00:00,  9.01s/it]


Processing complete!


## save results

In [124]:
from scipy import sparse
import scipy.io as sio

In [126]:
agg_mean.shape

(250669, 18085)

In [130]:
# gene names
sd.tables['square_002um'].var.loc[:, 'gene_ids'].to_csv(save_path + sample_name + '_gene_names.csv', sep=',', header=True, index=True)

# save merged expression matrix
sio.mmwrite(save_path + sample_name + '_merged_exp.csv', sparse.csr_matrix(agg_mean))

# save mean coordinates
np.savetxt(save_path + sample_name + '_mean_coordinates.csv', kernel_coords, delimiter=',')

# save count distribution
# NaN is first column, then 0, 1, 2, 3, ...
np.savetxt(save_path + sample_name + '_count_distribution.csv', count_dist, delimiter=',')